In [1]:
!pip install transformers datasets accelerate tqdm torch rouge-score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from sklearn.utils import shuffle
from datasets import load_dataset

naits_lfqa = load_dataset('fahdsoliman/naits_lfqa_processed')
old_lfqa = load_dataset('fahdsoliman/lfqa_with_supports_subset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/60 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/25 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/3004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/286 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/108 [00:00<?, ? examples/s]

In [3]:
naits_lfqa

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 60
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 25
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 17
    })
})

In [4]:
old_lfqa

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 3004
    })
    validation: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 286
    })
    test: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 108
    })
})

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from datasets import load_dataset, concatenate_datasets

# اختيار جزء من البيانات باستخدام `select`
old_lfqa_train_split = old_lfqa['train'].shuffle().select(range(60))
old_lfqa_val_split = old_lfqa['validation'].shuffle().select(range(25))

# دمج البيانات
combined_data = {}
combined_data['train'] = concatenate_datasets([old_lfqa_train_split, naits_lfqa['train']])
combined_data['train'] = combined_data['train'].shuffle()
combined_data['validation'] = concatenate_datasets([old_lfqa_val_split, naits_lfqa['validation']])
combined_data['validation'] = combined_data['validation'].shuffle()


In [ ]:
combined_data

{'train': Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 240
 }),
 'validation': Dataset({
     features: ['question', 'context', 'answer'],
     num_rows: 100
 })}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorWithPadding, get_scheduler, BartConfig
from torch.utils.data import DataLoader
from accelerate import Accelerator
from accelerate.utils import set_seed
import torch
# loading LFQA model

model_name = "vblagoje/bart_lfqa"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [ ]:
import numpy as np
label_pad_token_id = -100
def tokenize_dataset(examples):
    inputs = ["question: {} context: {}".format(q, c) for q, c in zip(examples["question"], examples["context"])]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=1024, padding='max_length', truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=360, padding='max_length', truncation=True,
                            return_tensors="np")

    model_inputs["decoder_input_ids"] = labels["input_ids"][:, :-1].tolist()
    # replace pad_token_id with label_pad_token_id to avoid loss calculation on those tokens
    labels["input_ids"] = np.where(labels["input_ids"] == tokenizer.pad_token_id,
                                    label_pad_token_id, labels["input_ids"])

    model_inputs["labels"] = labels["input_ids"][:, 1:].tolist()
    return model_inputs

tokenized_datasets = {}
for split in ["train", "validation"]:
        tokenized_datasets[split] = combined_data[split].map(tokenize_dataset,
                                                    batched=True,
                                                    remove_columns=combined_data[split].column_names,
                                                    desc="Preparing dataset for tokenization")

Preparing dataset for tokenization:   0%|          | 0/120 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Preparing dataset for tokenization:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets['train'][0])

{'input_ids': [0, 40018, 35, 16, 5, 6872, 2086, 2862, 15, 143, 31087, 672, 116, 5377, 35, 28696, 642, 15698, 5, 2771, 1683, 16, 10, 1161, 872, 9, 1468, 6, 2319, 23, 228, 76, 4, 42, 1280, 16, 9, 5, 746, 6872, 2862, 4, 5, 1013, 1161, 872, 16, 5700, 528, 7, 727, 6, 151, 7741, 685, 528, 7, 24486, 5111, 6, 8, 41, 674, 9, 2248, 6, 151, 7741, 3491, 31, 11, 12, 7349, 154, 8402, 8, 12505, 5110, 4, 42, 16, 157, 624, 5, 2862, 4983, 9, 321, 4, 2663, 207, 49038, 98, 5, 2319, 923, 9, 6872, 18, 2862, 16, 32512, 30, 42, 3724, 4, 50118, 28696, 642, 15698, 24, 16, 10, 157, 684, 754, 14, 5, 6872, 16, 5861, 6819, 2862, 149, 20477, 9, 10889, 8, 8402, 31, 980, 6, 25, 32, 70, 97, 33417, 3738, 11, 84, 467, 4, 309, 7, 295, 8810, 6, 22, 22466, 183, 59, 727, 7741, 9, 12505, 22926, 480, 27958, 9, 8402, 8, 26923, 8, 2128, 190, 380, 10889, 126, 2914, 5, 6872, 18, 5466, 72, 5, 1647, 9, 42, 8824, 14827, 62, 11, 5, 5466, 8, 8952, 25, 8402, 4, 215, 7678, 32695, 6, 959, 6, 16, 129, 10, 5251, 20214, 7590, 13484, 9, 5, 28

In [ ]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
train_dataset.set_format(type='torch')
eval_dataset.set_format(type='torch')

In [ ]:
from datasets import concatenate_datasets

preprocessed_dataset = concatenate_datasets([naits_lfqa['test'], old_lfqa['test']])
preprocessed_dataset2 = old_lfqa['test']


In [ ]:
# evaluate function

from datasets import load_metric, load_dataset
from tqdm.auto import tqdm
import torch


def evaluate_model(preprocessed_dataset):
    metric_rouge = load_metric("rouge", trust_remote_code=True)
    metric_bleu = load_metric("sacrebleu", trust_remote_code=True)
    all_predictions = []
    all_labels = []
    model.eval()

    for example in tqdm(preprocessed_dataset):
        # إعداد المدخلات
        inputs = tokenizer("question: {} context: {}".format(example["question"], example["context"]), truncation=True, padding=True, return_tensors="pt")
        # print(inputs)
        reference = example["answer"]

        # توليد التنبؤات
        with torch.no_grad():
            outputs = model.generate(input_ids=inputs["input_ids"].to(device),
                                    attention_mask=inputs["attention_mask"].to(device),
                                    min_length=64,
                                    max_length=256,
                                    do_sample=False,
                                    early_stopping=True,
                                    num_beams=8,
                                    temperature=1.0,
                                    top_k=None,
                                    top_p=None,
                                    eos_token_id=tokenizer.eos_token_id,
                                    no_repeat_ngram_size=3,
                                    num_return_sequences=1)

        prediction = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)

        # تجميع التنبؤات والمراجع
        all_predictions.append(prediction)
        all_labels.append(reference)

    # حساب ROUGE
    rouge_result = metric_rouge.compute(predictions=all_predictions, references=all_labels, use_stemmer=True)
    rouge_scores = {key: value.mid.fmeasure * 100 for key, value in rouge_result.items()}

    # حساب BLEU
    bleu_result = metric_bleu.compute(predictions=all_predictions, references=[[ref] for ref in all_labels])
    bleu_score = bleu_result["score"]

    print("ROUGE Scores:", rouge_scores)
    print("BLEU Score:", bleu_score)

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

# إنشاء DataCollator لتجميع البيانات
data_collator = DataCollatorWithPadding(tokenizer)

# إعداد DataLoader للتدريب والاختبار
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=False, batch_size=6, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], shuffle=False, batch_size=6, collate_fn=data_collator)

# إعداد النموذج والتدريب باستخدام Accelerator
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=0.0), train_dataloader, eval_dataloader
)

# جدولة التعلم
num_training_steps = len(train_dataloader) * 3  # عدد العصور
print(num_training_steps)
num_warmup_steps = int(0.05 * num_training_steps)
print(num_warmup_steps)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)


60
3


In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,  # Set this to a larger number, early stopping will prevent overfitting
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-6,
    weight_decay=0.01,
    evaluation_strategy="steps",  # Evaluate at every eval_steps
    eval_steps=100,  # Adjust this based on your dataset size
    save_strategy="steps",
    save_steps=100,  # Save at the same frequency as evaluation
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Use eval_loss as the metric to monitor
    greater_is_better=False,  # Lower loss is better
)


In [ ]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # Number of evaluations with no improvement after which training will be stopped
    early_stopping_threshold=0.01  # Minimum change to qualify as an improvement
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping_callback]  # Add the early stopping callback here
)

In [ ]:
trainer.train()

In [ ]:
from huggingface_hub import Repository
from tqdm.auto import tqdm

# إعداد المستودع لرفع النموذج
repo_name = "fahdsoliman/bart_lfqa_naits_subset_mixed"  # اسم النموذج على Huggingface Hub
repo = Repository(local_dir="model_repo", clone_from=repo_name)

set_seed(42)
progress_bar = tqdm(range(num_training_steps), disable=not accelerator.is_local_main_process)
# تدريب النموذج
model.train()
for epoch in range(3):
    train_losses = []
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        train_losses.append(loss.item())
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        progress_bar.set_postfix(loss=round(loss.item(), 3))

    # حساب وعرض متوسط الخسارة لكل دورة تدريبية
    average_train_loss = sum(train_losses) / len(train_losses)
    print(f"Epoch {epoch + 1} Train Loss: {average_train_loss:.4f}")

    # تقييم النموذج على مجموعة التحقق
    model.eval()
    eval_losses = []  # قائمة لتخزين الخسائر لكل دفعة خلال التقييم
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        eval_losses.append(outputs.loss.item())  # تخزين قيمة الخسارة للدفعة الحالية

    # حساب وعرض متوسط الخسارة لكل دورة تدريبية
    average_eval_loss = sum(eval_losses) / len(eval_losses)
    print(f"Epoch {epoch + 1} Validation Loss: {average_eval_loss:.4f}")

    evaluate_model(preprocessed_dataset2)
    evaluate_model(preprocessed_dataset)
    # العودة لوضع التدريب للمواصلة في الدورة التالية
    model.train()

    # حفظ النموذج في نهاية كل عصر ورفع إلى Huggingface Hub
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained("model_repo", save_function=accelerator.save)
    tokenizer.save_pretrained("model_repo")
    # repo.push_to_hub(commit_message=f"Epoch {epoch + 1} completed")

    print(f"Epoch {epoch + 1} completed.")

# حفظ النموذج النهائي ورفعه إلى Huggingface Hub
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained("model_repo", save_function=accelerator.save)
tokenizer.save_pretrained("model_repo")
# repo.push_to_hub(commit_message="Final model after training")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/fahdsoliman/bart_lfqa_naits_subset_mixed into local empty directory.


  0%|          | 0/60 [00:00<?, ?it/s]

Epoch 1 Train Loss: 2.6765
Epoch 1 Validation Loss: 2.7629


<ipython-input-9-593f2c4d81ba>:9: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_rouge = load_metric("rouge", trust_remote_code=True)


  0%|          | 0/108 [00:00<?, ?it/s]

...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


ROUGE Scores: {'rouge1': 29.418487714989862, 'rouge2': 7.558513410686799, 'rougeL': 16.56929715123299, 'rougeLsum': 16.5828198308545}
BLEU Score: 1.6774252520296522


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


Epoch 3 completed.


('model_repo/tokenizer_config.json',
 'model_repo/special_tokens_map.json',
 'model_repo/vocab.json',
 'model_repo/merges.txt',
 'model_repo/added_tokens.json',
 'model_repo/tokenizer.json')

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
# it all starts with a question/query
query = "If I do not pay the service fees within a month and the service is temporarily suspended, can I obtain the information I have on the hosting service?"

# given the question above suppose these documents below were found in some document store
documents = ["Article (45) fees: Hosting service fees are collected according to the fees approved by the Authority. Article (46) fees: The Authority has the right to amend the fees for the hosting service, and the new fees for the service are applied upon the first renewal or request for modification of the current hosting by the customer. Article (47) fees The Authority adopts a pre-payment system for all fees related to the hosting service, Article (48) fees: Public sector entities are excluded from the provisions of Article 44 in this document, as the Authority sends a financial claim after booking the hosting to these entities, provided that the payment notification is received within One month from the date of reservation, Article (49) Fees: The customer can pay location hosting fees every six months, provided that this is mentioned in the location hosting contract.",
             ]



# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)


["If you do not pay the service fees within a month and the service is temporarily suspended, you can obtain the information you have on the hosting service, provided that this is mentioned in the location hosting contract. If you don't pay the fee within the period of the suspension, you may not be able to get the information."]

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# it all starts with a question/query
query = "Why does water heated to room temperature feel colder than the air around it?"

# given the question above suppose these documents below were found in some document store
documents = ["when the skin is completely wet. The body continuously loses water by...",
             "at greater pressures. There is an ambiguity, however, as to the meaning of the terms 'heating' and 'cooling'...",
             "are not in a relation of thermal equilibrium, heat will flow from the hotter to the colder, by whatever pathway...",
             "air condition and moving along a line of constant enthalpy toward a state of higher humidity. A simple example ...",
             "Thermal contact conductance In physics, thermal contact conductance is the study of heat conduction between solid ..."]

# concatenate question and support documents into BART input
conditioned_doc = "<P> " + " <P> ".join([d for d in documents])
query_and_docs = "question: {} context: {}".format(query, conditioned_doc)

model_input = tokenizer(query_and_docs, truncation=True, padding=True, return_tensors="pt")

generated_answers_encoded = model.generate(input_ids=model_input["input_ids"].to(device),
                                           attention_mask=model_input["attention_mask"].to(device),
                                           min_length=64,
                                           max_length=256,
                                           do_sample=False,
                                           early_stopping=True,
                                           num_beams=8,
                                           temperature=1.0,
                                           top_k=None,
                                           top_p=None,
                                           eos_token_id=tokenizer.eos_token_id,
                                           no_repeat_ngram_size=3,
                                           num_return_sequences=1)
tokenizer.batch_decode(generated_answers_encoded, skip_special_tokens=True,clean_up_tokenization_spaces=True)



["The air around the water feels colder than the air around it. This is due to the difference in temperature between the water and the air. The air is moving faster than the water, so when the water is heated, the air moves slower, so it feels colder. It's the same reason why you feel cold when you're in a hot car, even though it's not actually that cold."]